In [66]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
import scipy.io as sio
import tools.graph_processing as gp
import scipy.stats as stats
import math

In [2]:
path_to_labelled_graphs = '../data/Oasis_original_new_with_dummy/labelled_graphs'
path_ro_correspondence = 'gender_correspondence.pickle'

In [3]:
def create_clusters_lists_with_label_gender(list_graphs,gender_corresp,label_attribute="label_dbscan"):

    result_dict = {}
    label_depths = {}
    label_gender = {}

    for i_graph, graph in enumerate(list_graphs):
        for node in graph.nodes:
            if not graph.nodes[node]["is_dummy"]:
                label_cluster = graph.nodes[node][label_attribute]
                
                if label_cluster in result_dict:
                    
                    #retrieve depth of the corresponding label in that graph
                    depth_value = graph.nodes[node]['depth']
                    
                    result_dict[label_cluster].append((i_graph, node))
                    label_depths[label_cluster].append(depth_value)
                    label_gender[label_cluster].append(gender_corresp[i_graph])
                    
                else:
                    #retrieve depth of the corresponding label in that graph
                    depth_value = graph.nodes[node]['depth']
                    
                    result_dict[label_cluster] = [(i_graph, node)]
                    label_depths[label_cluster] = [depth_value]
                    label_gender[label_cluster] = [gender_corresp[i_graph]]


    return result_dict,label_depths,label_gender

In [13]:
labeled_graphs = gp.load_graphs_in_list(path_to_labelled_graphs)
gender_corresp = np.array(nx.read_gpickle(path_ro_correspondence))[:,2]# gender correp list

In [20]:
def seperate_groups_by_label(label_gender,label_depths):
    
    # Separate groups by label
    
    label_gen_sep = []
    for key in label_gender.keys():
        M = []
        F = []
        for i in range(len(label_gender[key])):

            if label_gender[key][i] == 'F':

                F.append(label_depths[key][i])
            else:
                M.append(label_depths[key][i])
                
         # 1st list M, 2nd F
        
        label_gen_sep.append([M,F])
        
    return label_gen_sep
    

In [21]:
def calculate_tstats_and_pvalues(corresp ,method = 'labelling_mALS'):
    
    # get labeled groups and depths    
    result_dict,label_depths,label_gender = create_clusters_lists_with_label_gender(labeled_graphs,corresp,method)
    
    # depth seperated by groups
    label_gen_sep = seperate_groups_by_label(label_gender,label_depths)
    
    t_stats = {}

    for key,lst in zip(label_gender.keys(),label_gen_sep):

        res = stats.ttest_ind(a=lst[0], b=lst[1], equal_var=True)

        t_stats[key] = [res[0],res[1]]
        
    return t_stats

In [23]:
calculate_tstats_and_pvalues(gender_corresp, 'labelling_mSync')

{0.6726961301647116: [0.18400002454622633, 0.854313995948472],
 0.3119287115852154: [-1.7661376047780655, 0.07993712327843999],
 0.6011496294320424: [0.40035153399714296, 0.68996471951317],
 0.8893209701421761: [-1.5942592007080807, 0.11344443744693955],
 0.8805262168776856: [-4.022193168598675, 9.651301335327734e-05],
 0.055688767098739926: [1.2320633899523332, 0.22008296934212607],
 0.05871240825451698: [-0.9534996346067645, 0.3474845241820387],
 0.4325710901301789: [-1.3716397671315403, 0.17296689409062158],
 0.6731787537133088: [-0.7383439371239279, 0.4615870963591858],
 0.5614567581322376: [-0.9227113425577724, 0.3578038149391024],
 0.3369472456986446: [-0.5167776606174213, 0.6062101385997775],
 0.8591421507706923: [1.6566920395715385, 0.10317486878828541],
 0.7943269487351451: [0.6396893514962055, 0.5234917042625473],
 0.48756164187770873: [-0.48456648670238617, 0.6288598979409452],
 0.6865387024115662: [-0.19331037041298016, 0.8471243131701328],
 0.3264752693192501: [-1.05823122

In [27]:
calculate_tstats_and_pvalues(gender_corresp, 'labelling_MatchEig')

{0.6726961301647116: [0.029602932472483058, 0.9764383617137351],
 0.3119287115852154: [-2.5027264949503607, 0.013604021949527355],
 0.8673323145184816: [-0.7604085881939685, 0.4486827447150805],
 0.8893209701421761: [-1.4095713432206278, 0.16107256980902554],
 0.6865387024115662: [-1.6568834045906926, 0.10183524543752848],
 0.055688767098739926: [1.1240943337104592, 0.26306042605143987],
 0.05871240825451698: [-1.0533796316965065, 0.2941209748775465],
 0.4325710901301789: [0.23503682499863637, 0.8145557436901087],
 0.6731787537133088: [-1.230173220242283, 0.22082011520530323],
 0.5614567581322376: [0.1887396554392283, 0.8505889544032117],
 0.3369472456986446: [-0.2671801843732027, 0.790001252901495],
 0.8591421507706923: [-0.7011335974867164, 0.48466100027482073],
 0.7943269487351451: [0.6124915916748414, 0.5413076980647734],
 0.48756164187770873: [0.1941326580632268, 0.8464333518297618],
 0.12357520989808803: [0.051374288735794166, 0.9594908869389727],
 0.3264752693192501: [-1.1011890

In [33]:
calculate_tstats_and_pvalues(gender_corresp, 'labelling_mALS')

{0.6726961301647116: [-0.9393205500309549, 0.35281295936829393],
 0.3119287115852154: [-1.44107245676289, 0.15378245368357538],
 -0.1: [-1.7981280518084264, 0.07224421166239267],
 0.8893209701421761: [-1.7039740411943542, 0.09236691937837412],
 0.055688767098739926: [0.558259915184414, 0.5776345606562069],
 0.6164804022678066: [0.6588713130945588, 0.5113187568232636],
 0.5614567581322376: [-0.7126778979108633, 0.47758203659844667],
 0.7943269487351451: [0.9360865602275951, 0.3515533307185369],
 0.48756164187770873: [-1.1160933933987816, 0.267682220926069],
 0.8123777065116572: [-0.5689294812347238, 0.5704087589139389],
 0.676948807071108: [1.2390929376625963, 0.21969951429831175],
 0.879479847512742: [0.5872884821216837, 0.5580693016068246],
 0.5376379964782969: [-1.363298686457699, 0.17509175858469317],
 0.28967332925889594: [0.3514605529101189, 0.7258118561264489],
 0.956799027313698: [1.117657239640099, 0.26594602871587897],
 0.5231860053967116: [0.8506075203708285, 0.39652773410851

In [32]:
labeled_graphs[0].nodes.data()[0]

{'vertex_index': 1357,
 'sphere_3dcoords': array([  6.50138903, -93.98175049, -33.54344177]),
 'sphere_coords': [-1.5017292499542236, 1.9128626585006714],
 'basin_label': 307.0,
 'basin_area': 447.01047,
 'depth': -0.18439683,
 'basin_thickness': 2.5869646072387695,
 'ico100_7_vertex_index': 7574,
 'label_neuroimage': 28,
 'label_media': 90,
 'sphere_3dcoords_noreg': array([ 17.402021, -91.770744, -35.71135 ], dtype=float32),
 'ico100_7_vertex_index_noreg': 11492,
 'is_dummy': False,
 'labelling_kerGM': 0.6726961301647116,
 'labelling_mALS': 0.6726961301647116,
 'labelling_mSync': 0.6726961301647116,
 'labelling_CAO': 0.2269832262372122,
 'labelling_MatchEig': 0.6726961301647116}

In [68]:
tstat_mSync = calculate_tstats_and_pvalues(gender_corresp, 'labelling_mSync')

In [49]:
kk = np.array(list(tstat_mALS.values()))[:,1]
kk

array([3.52812959e-01, 1.53782454e-01, 7.22442117e-02, 9.23669194e-02,
       5.77634561e-01, 5.11318757e-01, 4.77582037e-01, 3.51553331e-01,
       2.67682221e-01, 5.70408759e-01, 2.19699514e-01, 5.58069302e-01,
       1.75091759e-01, 7.25811856e-01, 2.65946029e-01, 3.96527734e-01,
       8.57883258e-04, 8.80491270e-01, 4.01095558e-01, 5.89814979e-01,
       6.34549430e-01, 3.88864898e-01, 2.69557050e-02, 1.32043381e-01,
       5.68559008e-01, 3.86716142e-01, 3.62582993e-01, 4.99617905e-01,
       8.63618664e-01, 6.25834194e-01, 8.59537658e-01, 9.24020083e-01,
       8.91212376e-02, 5.10286329e-01, 8.74359636e-01, 3.10521930e-01,
       1.85724870e-01, 9.92261488e-01, 5.64042523e-01, 7.12845782e-01,
       7.97683116e-01, 1.53426685e-01, 2.08258654e-01, 7.63448994e-01,
       8.85546693e-01, 3.51699102e-01, 6.98619426e-01, 7.53467597e-01,
       5.23577006e-01, 1.04918892e-02, 9.12548726e-01, 2.59479718e-01,
       9.80740239e-01, 3.27857972e-01, 8.12055251e-01, 7.28965362e-02,
      

In [80]:
tstat_mALS_copy = tstat_mALS.copy()
tstat_mSync_copy = tstat_mSync.copy()

In [56]:
# result_dict,label_depths,label_gender = create_clusters_lists_with_label_gender(labeled_graphs,gender_corresp,'labelling_mSync')

In [91]:
def drop_nan(t_stat_dict):
    
    drop_k = []
    for k in t_stat_dict.keys():
        if math.isnan(t_stat_dict[k][0]):
            drop_k.append(k)

    for k in drop_k:
        t_stat_dict.pop(k)
    
    return np.array(list(t_stat_dict.values()))

In [90]:
drop_nan(tstat_mSync_copy)

array([ 0.18400002, -1.7661376 ,  0.40035153, -1.5942592 , -4.02219317,
        1.23206339, -0.95349963, -1.37163977, -0.73834394, -0.92271134,
       -0.51677766,  1.65669204,  0.63968935, -0.48456649, -0.19331037,
       -1.05823123, -0.02644108, -0.406177  , -0.49970162, -0.19765852,
        0.56926528, -0.44938764,  0.45455372, -0.33520952, -1.1699983 ,
       -1.02581672,  0.043568  ,  1.62176566, -0.69128702, -3.03088198,
        1.69105499, -1.13335023,  0.7268444 , -0.69841202, -1.36929406,
        0.25566427, -0.52740943, -1.32007473, -2.45880345, -0.42165979,
        0.11584227, -0.57157699, -0.27691301,  0.91354814, -0.36739416,
       -0.4383382 , -0.89856691,  0.29539839, -2.7352511 ,  0.67669682,
        0.46886829, -0.8111069 , -0.12312088,  0.24275047, -0.80192745,
       -1.53866992, -0.74669999, -0.49806652, -0.5718094 ,  1.35909663,
        2.512246  , -0.54022976, -0.53635429,  0.11895098,  0.27973217,
       -2.27310232,  0.05224738,  0.76147209, -0.61518613,  1.00